In [1]:
import numpy as np

# Define the path to your embeddings file
embeddings_file_path = r"C:\Users\Anuz\OneDrive\Desktop\excel work\Embeddings.txt"

# Initialize an empty dictionary to store embeddings
embeddings = {}

# Read the embeddings from the file
with open(embeddings_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()  # Split line into word and vector components
        word = parts[0]  # The first part is the word
        vector = np.array(parts[1:], dtype=float)  # Convert the rest to a NumPy array of floats
        embeddings[word] = vector  # Store in the dictionary

print(f"Loaded {len(embeddings)} word embeddings.")

Loaded 43047 word embeddings.


In [2]:
import pandas as pd

# Define the path to your tokenized data file
tokenized_data_file_path = r"C:\Users\Anuz\OneDrive\Desktop\excel work\stemmed_tokenized_cleaned_dataset.csv"

# Load the tokenized data from CSV file
df = pd.read_csv(tokenized_data_file_path)

# Convert string representation of lists back to actual lists
tokenized_sentences = df['stemmed_text'].apply(eval).tolist()

print(f"Loaded {len(tokenized_sentences)} tokenized sentences.")

Loaded 8639 tokenized sentences.


In [3]:
# Create a vocabulary mapping from words to indices
word_to_index = {word: idx for idx, word in enumerate(embeddings.keys())}
index_to_word = {idx: word for word, idx in word_to_index.items()}

print(f"Vocabulary size: {len(word_to_index)}")
vocab_dict = word_to_index

Vocabulary size: 43047


In [4]:
# import torch
# import torch.nn as nn
# max_seq_len = 200  # Maximum length of input sequences
# padding_idx = len(word_to_index)  # Use this index for padding (if needed)

# sequences_data = []

# for tokens in tokenized_sentences:
#     # Create a padded sequence initialized with padding index
#     padded_sequence = [padding_idx] * max_seq_len
    
#     # Fill in the indices corresponding to words found in word_to_index
#     for i, tok in enumerate(tokens):
#         if tok in word_to_index:
#             if i < max_seq_len:  # Ensure we do not exceed max_seq_len
#                 padded_sequence[max_seq_len - i - 1] = word_to_index[tok]  # Fill from the end

#     sequences_data.append(padded_sequence)

# # Convert to tensors
# sequence_tensors = [torch.tensor(seq, dtype=torch.long) for seq in sequences_data]

In [5]:
# max_seq_len = 200  # Maximum length of input sequences
# padding_idx = 0     # Index for padding (if needed)

# # Prepare sequences and labels (you'll need actual target summaries here)
# sequences_data = []
# labels_data = []  # This should contain your target summaries

# for text in df['stemmed_text']:
#     tokens = text.split()
    
#     padded_sequence = [padding_idx]*max_seq_len
    
#     for i,tok in enumerate(tokens):
#         if tok in vocab_dict:
#             padded_sequence[max_seq_len-i-1]=vocab_dict[tok]  # Fill from the end
            
#     sequences_data.append(padded_sequence)

# # Convert to tensors
# sequence_tensors = [torch.tensor(seq, dtype=torch.long) for seq in sequences_data]

In [6]:
# class SummarizationModel(nn.Module):
#     def __init__(self, embedding_dim, hidden_dim, vocab_size):
#         super(SummarizationModel, self).__init__()
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, vocab_size)  # Ensure vocab_size matches

#     def forward(self, x):
#         lstm_out, _ = self.lstm(x)
#         out = self.fc(lstm_out[:, -1, :])  # Use output from last time step
#         return out

# # Define parameters for the model
# embedding_dim = 100  # This should match your GloVe embedding size
# hidden_dim = 128     # You can adjust this based on your architecture
# vocab_size = len(word_to_index) + 1  # Adding 1 for padding index

# # Create an instance of the model
# model = SummarizationModel(embedding_dim=embedding_dim, hidden_dim=hidden_dim, vocab_size=vocab_size)

In [7]:
# #defining custom dataset class 
# from torch.utils.data import Dataset, DataLoader

# class NewsDataset(Dataset):
#     def __init__(self, sequences):
#         self.sequences = sequences

#     def __len__(self):
#         return len(self.sequences)

#     def __getitem__(self, idx):
#         return self.sequences[idx]

# # Create dataset and dataloader
# dataset = NewsDataset(sequence_tensors)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [8]:
# #training function for lstm model 
# def train_model(model, dataloader, epochs=10):
#     model.train()  # Set model to training mode
    
#     criterion = nn.CrossEntropyLoss()  # Loss function
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Optimizer
    
#     for epoch in range(epochs):
#         total_loss = 0
        
#         for inputs in dataloader:
#             optimizer.zero_grad()  # Clear previous gradients
            
#             outputs = model(inputs)  # Get predictions from the model
            
#             loss = criterion(outputs.view(-1, outputs.size(-1)),targets.view(-1), inputs.view(-1))  # Compute loss
            
#             loss.backward()  # Backpropagation step
            
#             optimizer.step()  # Update weights
            
#             total_loss += loss.item()
        
#         print(f'Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(dataloader):.4f}')

In [16]:
#implementing the textrank algorithm
from sklearn.metrics.pairwise import cosine_similarity

def get_sentence_vector(sentence, embeddings):
    tokens = sentence.split()  # Tokenize the sentence into words
    vectors = [embeddings[word] for word in tokens if word in embeddings]
    
    # Return a zero vector if no valid tokens are found
    if not vectors:  
        return np.zeros(100)  # Return a zero vector of embedding size (100)
    
    return np.mean(vectors, axis=0)  # Average out the vectors
def textrank(sentences, embeddings):
    sentence_vectors = []
    
    for sentence in sentences:
        vector = get_sentence_vector(sentence, embeddings)
        # print(f"Sentence: {sentence}")  # Print the sentence
        # print(f"Vector shape: {vector.shape}")  # Print the shape of the vector
        sentence_vectors.append(vector)

    #Convert list of vectors to a NumPy array
    try:
        sentence_vectors = np.array(sentence_vectors)
    except ValueError as e:
        print("Error creating NumPy array:", e)
        print("Sentence Vectors:", sentence_vectors)  # Debugging output
        return []

    # Compute similarity matrix
    similarity_matrix = cosine_similarity(sentence_vectors)

    # Rank sentences based on scores (sum of similarities)
    scores = np.sum(similarity_matrix, axis=1)
    ranked_sentences = [sentences[i] for i in np.argsort(scores)[::-1]]  # Sort sentences by score
    
    return ranked_sentences[:3]  # Return top 3 sentences as summary

In [17]:
test_article = """सताब्दियौंदेखि संसारभर कागजी नोट तथा सिक्का (भौतिक मुद्रा) प्रचलनमा छन् । पछिल्लो समय अभौतिक मुद्राको विकासमा विश्वका धेरै मुलुकहरुले काम गरिरहेका छन् ।

एक अध्ययनले देखाए अनुसार विश्व अर्थतन्त्रमा ९८ प्रतिशत योगदान रहेका देशका केन्द्रीय बैंकले ‘सेन्ट्रल बैंक डिजिटल करेन्सी’ (सीबीडीसी) विकास गर्न काम गरिरहेका छन् ।

सीबीडीसी भनेको भौतिक मुद्रालाई अभौतिक (डिजिटल) रुपमा प्रचलनमा ल्याउने प्रणाली हो । यसको निष्कासन नै अभौतिक रुपमा हुन्छ । भौतिक मुद्रा जस्तै सीबीडीसी पनि विभिन्न किसिमका हुन्छन् । यसको नियन्त्रण केन्द्रीय बैंकबाट नै हुन्छ । यसको कानुनी अधिकार केन्द्रीय बैंकसँग हुन्छ । यसको दायित्व पनि केन्द्रीय बैंकको नै हुन्छ ।

अहिले केन्द्रीय बैंकले नोट निष्कासन गर्छ । साथै इ–मनिको रुपमा मोबाइल बैंकिङ, मोबाइल वालेट लगायतको डिजिटल भुक्तानी उपकरण मार्फत कारोबार हुँदै आएको छ । यसको ‘ब्याकअप’ मा बैकहरुसँग नोट नै रहन्छ ।

सेन्ट्रल बैंक डिजिटल करेन्सीको पनि ‘केन्द्रीय बैंक’ नै त्यसको ज्ञारेन्टीको रुपमा रहन्छ । खुद्रा सीबीडीसी भएमा त्यसको ‘ब्याकअप’मा बैंक वा केन्द्रीय बैंक हुन्छ । अहिले मोबाइल बैंकिङ, इन्टरनेट बैंकिङ, क्युआर कोडको कारोबार तत्काल भएपनि त्यसको सेटलमेन्ट भने १/२ दिनमा मात्रै हुन्छ ।

सीबीडीसी रियल टाइममा सेटलमन्ट हुन्छ । डिजिटल रुपमा मुद्रा निष्कासन गर्ने र त्यसको दायित्व लिनेगरी ‘ब्लकचेन प्रविधि’ मा आधारित रहेर केन्द्रिकृत रुपमा मुद्राको कारोबार गर्न अनुमति प्रदान गरिन्छ भने त्यसलाई सीबीडीसी भनिन्छ ।

विश्वव्यापी रुपमा सीबीडीसीको सुरुवात र तीव्र विकास कोभिड सँगसँगै भएको देखिन्छ । त्यसअघि नै ब्लकचेन प्रविधिमा आधारित भएर क्रिप्टोरकरेन्सी तीव्र विकास भइसकेको थियो । त्यसले परम्परागत कागजी मुद्रामाथि चुनौती थपेकै थियो ।"""

# Get extractive summary using TextRank 
extractive_summary = textrank(test_article.split('।'), embeddings)
print("Extractive Summary:", '।'.join(extractive_summary))

Extractive Summary: 

अहिले केन्द्रीय बैंकले नोट निष्कासन गर्छ । खुद्रा सीबीडीसी भएमा त्यसको ‘ब्याकअप’मा बैंक वा केन्द्रीय बैंक हुन्छ ।

एक अध्ययनले देखाए अनुसार विश्व अर्थतन्त्रमा ९८ प्रतिशत योगदान रहेका देशका केन्द्रीय बैंकले ‘सेन्ट्रल बैंक डिजिटल करेन्सी’ (सीबीडीसी) विकास गर्न काम गरिरहेका छन् 


In [11]:
# def synonym_replacement(sentence):
#     synonyms = {
#              "सरकार": "सरकारी निकाय",
#     "बजेट": "आर्थिक योजना",
#     "बेरुजु": "अवशेष",
#     "खर्च": "व्यय",
#     "राजस्व": "आम्दानी",
#     "लेखा परीक्षण": "लेखा परिक्षण",
#     "महालेखा नियन्त्रक": "लेखा प्रमुख",
#     "वित्तीय वर्ष": "आर्थिक वर्ष",
#     "पेस्की": "अग्रिम रकम",
#     "अनुदान": "सहयोग राशि",
#     "ऋण": "कर्ज़",
#     "चक्रवृद्धि ब्याज": "चक्र ब्याज",
#     "निवेश": "लगानी",
#     "वित्तीय विवरण": "आर्थिक रिपोर्ट",
#     "सम्पत्ति": "सम्पत्ति साधन",
#     "दायित्व": "जिम्मेवारी",
#     "लाभांश": "मुनाफा वितरण",
#     "बजार मूल्य": "बजार दर",
#     "शेयर बजार": "शेयर व्यापार बजार",
#     "वित्तीय संस्थान": "बैंकिंग संस्था",
#     "कर प्रणाली": "कर संरचना",
#     "बचत खाता": "बचत खाता खाता",
#     "चालू खाता": "संचालन खाता",
#     "पूँजी बजार": "पूँजी व्यापार बजार",
#     "ऋणपत्र": "कर्ज़ पत्रिका",
#     "मूल्यांकन रिपोर्ट": "मूल्यांकन विवरणिका",
#     "वित्तीय योजना": "आर्थिक योजना",
#     "समग्र घरेलु उत्पादन": "कुल घरेलु उत्पादन (GDP)",
#     "विदेशी मुद्रा भण्डार": "अन्तर्राष्ट्रिय मुद्रा भण्डार",
#     "वित्तीय संकट": "आर्थिक संकट",
#     "स्थायी आय स्रोतहरू": "दीर्घकालीन आय स्रोतहरू",
#     "बिमा प्रिमियम":  "बिमा शुल्क",
#     "भुक्तानी प्रणाली":  "भुक्तानी विधि",
#     "उपभोक्ता मूल्य सूचकांक":  "उपभोक्ता मूल्य मापदण्ड",
#     "मुद्रा स्फीति":  "मूल्य वृद्धि",
#     "ऋण चुकाउने क्षमता":  "कर्ज़ चुकाउने क्षमता",
#     "आर्थिक विकास":  "वित्तीय प्रगति",
#     "बजेट अधिवेशन":  "बजेट बैठक",
#     "वित्तीय अनुगमन":  "आर्थिक निगरानी",
#     "व्यावसायिक योजना":  "व्यवसायिक रणनीति",
#     "सम्पत्ति व्यवस्थापन":  "सम्पत्ति नियन्त्रण",
#     "जोखिम व्यवस्थापन":  "जोखिम नियन्त्रण",
#     "वित्तीय अनुपात":  "आर्थिक अनुपात",
#     "आयकर":  "आम्दानी कर",
#     "सम्पत्ति कर":  "सम्पत्ति कराधान",
#     "बिदेशी लगानी":  "बाह्य लगानी",
#     "व्याज दर":  "ब्याजको दर",
#     "व्यापार घाटा":  "व्यापारको कमी",
#     "आर्थिक नीतिहरू":  "वित्तीय नीतिहरू",
#     "निवेश": "लगानी",
# "संचालन": "सञ्चालन",
# "वित्तीय": "आर्थिक",
# "उपाय": "उपाय",
# "स्रोत": "साधन",
# "समीक्षा": "परिक्षण",
# "सञ्चय": "बचत",
# "सहयोग": "समर्थन",
# "आवश्यकता": "आवश्यकता",
# "सम्भावना": "अवसर",
# "दिशा": "मार्गदर्शन",
# "बिक्री": "बिक्री",
# "प्रवृत्ति": "प्रवृत्ति",
# "संरचना": "ढाँचा",
# "अवधि": "समय सीमा",
# "प्रभाव": "असर",
# "सञ्चालन खर्च": "सञ्चालन व्यय",
# "आर्थिक नीति": "वित्तीय नीति",
# "बजार विश्लेषण": "बजार अध्ययन",
# "स्रोतहरू": "साधनहरू",
# "अर्थशास्त्र": "अर्थ विज्ञान",
# "प्रतिस्पर्धा": "प्रतिस्पर्धा",
# "लाभप्रदता": "मुनाफा बनाउने क्षमता",
# "सम्पर्क": "सम्बन्ध",
# "बिकास": "प्रगति",
# "परियोजना": "योजना",
# "संलग्नता": "संलग्नता",
# "सामग्री": "सामग्रीहरू",
# "पारदर्शिता": "स्पष्टता",
# "विकासशील देशहरू": "विकासशील राष्ट्रहरू",
# "आधारभूत संरचना": "आधारभूत ढाँचा",
# "साझेदारी": "सहकार्य",
# "सूचना प्रविधि": "सूचना टेक्नोलोजी",
# "व्यापारिक गतिविधि": "व्यापार क्रियाकलाप",
# "आर्थिक गतिविधि": "वित्तीय क्रियाकलाप",
# "पुनर्निर्माण गर्नुपर्छ": "सुधार गर्नुपर्छ",
# "उपाययोजना": "योजना बनाउनुपर्छ",
# "अभियान": "कार्यक्रम",
# "प्रतिबद्धता": "वचनबद्धता",
# "स्रोतको उपयोग": "साधनको प्रयोग",
# "लाभदायक": "फाइदा दिने",
# "बिकासशील": "प्रगतिशील",
# "अनुसन्धान": "अध्ययन",
# "प्रतिस्पर्धात्मक लाभ": "प्रतिस्पर्धात्मक फाइदा",
# "आधारभूत तत्वहरू": "मूल तत्वहरू",
# "व्यापारिक रणनीति": "व्यापार योजना",
# "उपभोक्ता व्यवहार": "ग्राहक व्यवहार"
#     }

#     for word, synonym in synonyms.items():
#         sentence = sentence.replace(word, synonym)

#     return sentence

# # Assuming extractive_summary is a list of sentences
# original_sentences = extractive_summary  # This should be your list of sentences
# modified_sentences = [synonym_replacement(sentence) for sentence in original_sentences]

# # If you want to join them back into a single string
# modified_text = ' '.join(modified_sentences)
# print(modified_text)